In [1]:
import os
import numpy as np
import pandas as pd

import mlflow
from mlflow.models import infer_signature

from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

In [2]:
os.getenv("MLFLOW_TRACKING_URI")

'http://mlflow-service:5000'

In [3]:
# Получим датасет California housing
housing = datasets.fetch_california_housing(as_frame=True)
# Объединим фичи и таргет в один np.array
data = pd.concat([housing["data"], pd.DataFrame(housing["target"])], axis=1)

In [4]:
FEATURES = [
    "MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup",
    "Latitude", "Longitude"
]
TARGET = "MedHouseVal"

In [5]:
# Сделать препроцессинг
# Разделить на фичи и таргет
X, y = data[FEATURES], data[TARGET]

# Разделить данные на обучение и тест
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Обучить стандартизатор на train
scaler = StandardScaler()
X_train_fitted = scaler.fit_transform(X_train)
X_test_fitted = scaler.transform(X_test)

# Обучить стандартизатор на train
scaler = StandardScaler()
X_train_fitted = scaler.fit_transform(X_train)
X_test_fitted = scaler.transform(X_test)

In [6]:
name = "MedHouseExp"
experiment_id = mlflow.create_experiment(name)
mlflow.set_experiment(experiment_id)

RestException: RESOURCE_ALREADY_EXISTS: Experiment 'MedHouseExp' already exists.

In [7]:
with mlflow.start_run(run_name="my_first_run", experiment_id = experiment_id):
    # Обучить модель
    model = LinearRegression()
    model.fit(X_train_fitted, y_train)
    y_pred = model.predict(X_test_fitted)

    # Получить описание данных
    signature = infer_signature(X_test_fitted, y_pred)
    # Сохранить модель в артифактори
    model_info = mlflow.sklearn.log_model(model, name, signature=signature)
    # Сохранить метрики модели
    mlflow.evaluate(
        model_info.model_uri,
        data=X_test_fitted,
        targets=y_test.values,
        model_type="regressor",
        evaluators=["default"],
    )

NameError: name 'experiment_id' is not defined